In [1]:
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [14]:
check = client.query(
"""
CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.adobe_carto_catalogue_homepage_ciid_search_term_test`  AS (
    with 
    
    cartology_campaign_asset_impressions_in_adobe AS (
        SELECT 
            date_time,
            shopper_id_map.shopper_id, 
            crn_map.crn, 
            post_evar6 AS page_name,
            CASE WHEN SPLIT(post_evar6,":")[SAFE_OFFSET(2)] = 'catalogue' OR LOWER(post_evar7) = 'catalogue' THEN 1 ELSE 0 END AS catalogue_flag,    
            CASE WHEN post_evar6 = 'ww-sm:homepage' THEN 1 ELSE 0 END AS homepage_flag, 
            CASE WHEN post_mvvar2 IS NOT NULL THEN SPLIT(post_mvvar2,":")[SAFE_OFFSET(0)] ELSE "Not Found" END AS booking_id,
            CASE WHEN post_mvvar2 IS NOT NULL THEN SPLIT(post_mvvar2,":")[SAFE_OFFSET(3)] ELSE NULL END AS ci_id,
            post_evar51 AS search_term1,
            post_evar52 AS search_term2,
            post_evar53 AS search_term3
            
        FROM `gcp-wow-food-wlx-digaspt-dev.prod_adobe_data.grs_adobe` adobe

        -- shopper id
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_shopperid_map_window` shopper_id_map
            ON adobe.prop9 = shopper_id_map.visitor_id
            AND adobe.date_time BETWEEN shopper_id_map.effective_time AND shopper_id_map.expiry_time
        
        -- crn
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_crn_map_window` crn_map
            ON adobe.prop9 = crn_map.visitor_id
            AND adobe.date_time BETWEEN crn_map.effective_time and crn_map.expiry_time

            

        WHERE 
            site_name = "Supermarkets"
        AND 
            DATE(date_time) >= DATE("2023-10-01")
        AND    
            DATE(date_time) < CURRENT_DATE("Australia/Sydney")
        AND         
            (
                -- Catalogue
                (SPLIT(post_evar6,":")[SAFE_OFFSET(2)] = 'catalogue' OR LOWER(post_evar7) = 'catalogue')
                OR 
                -- Homepage
                (post_evar6 = 'ww-sm:homepage') 
                OR 
                -- ci_id 
                (SPLIT(post_mvvar2,":")[SAFE_OFFSET(3)] IS NOT NULL)
            )
        AND      
            post_channel NOT LIKE "Order Confirmation Section"
        AND 
            (shopper_id_map.shopper_id IS NOT NULL OR crn_map.crn IS NOT NULL)
    
    ) 

    SELECT * FROM cartology_campaign_asset_impressions_in_adobe 

);
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

""


In [31]:
check = client.query(
"""
with step_one AS (
SELECT a.*, cc.media_type, cc.campaign_start_date, cc.campaign_end_date, cc.media_start_date, cc.media_end_date, dim.quoteline_sku
FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_carto_catalogue_homepage_ciid_search_term_test` a
LEFT JOIN `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc
    ON a.booking_id = cc.booking_number 
    AND a.ci_id = cc.line_name
LEFT JOIN `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim
    ON dim.booking_number = a.booking_id 
    AND dim.line_name = a.ci_id
WHERE cc.media_type IS NOT NULL
AND a.search_term1 IS NOT NULL
)
SELECT  
    booking_id, 
    campaign_start_date,
    campaign_end_date,
    media_start_date,
    media_end_date,
    media_type,
    ci_id, 
    search_term3,
    COUNT(DISTINCT search_term1) AS search_terms,
    ARRAY_AGG(DISTINCT sku ORDER BY sku) AS skus
    --search_term2,
    --search_term3,
FROM step_one,
UNNEST(SPLIT(quoteline_sku, ",")) AS sku
WHERE booking_id = "WOW20003368"
--WHERE LOWER(media_type) LIKE "%search%"
GROUP BY 1,2,3,4,5,6,7,8
ORDER BY media_start_date, ci_id
LIMIT 500
"""
).result()
check_df = check.to_dataframe()
check_df 

,booking_id,campaign_start_date,campaign_end_date,media_start_date,media_end_date,media_type,ci_id,search_term3,search_terms,skus
0,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120856,SuggestedV5,322,"[151099, 151702, 278262]"
1,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120856,None,18,"[151099, 151702, 278262]"
2,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120856,Self-Typed,883,"[151099, 151702, 278262]"
3,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120856,CorrectedV5,12,"[151099, 151702, 278262]"
4,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120856,SearchHistory,38,"[151099, 151702, 278262]"
5,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120857,None,34,"[151099, 151702, 278262]"
6,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120857,SearchHistory,29,"[151099, 151702, 278262]"
7,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120857,Self-Typed,1104,"[151099, 151702, 278262]"
8,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120857,SuggestedV5,342,"[151099, 151702, 278262]"
9,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120857,CorrectedV5,33,"[151099, 151702, 278262]"


In [33]:
check = client.query(
"""
with step_one AS (
SELECT a.*, cc.media_type, cc.campaign_start_date, cc.campaign_end_date, cc.media_start_date, cc.media_end_date, dim.quoteline_sku
FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_carto_catalogue_homepage_ciid_search_term_test` a
LEFT JOIN `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc
    ON a.booking_id = cc.booking_number 
    AND a.ci_id = cc.line_name
LEFT JOIN `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim
    ON dim.booking_number = a.booking_id 
    AND dim.line_name = a.ci_id
WHERE cc.media_type IS NOT NULL
AND a.search_term1 IS NOT NULL
)
SELECT  
    booking_id, 
    campaign_start_date,
    campaign_end_date,
    media_start_date,
    media_end_date,
    media_type,
    ci_id, 
    search_term3,
    COUNT(DISTINCT search_term1) AS n_search_terms,
    ARRAY_AGG(DISTINCT search_term1 ORDER BY search_term1) AS search_terms,
    ARRAY_AGG(DISTINCT sku ORDER BY sku) AS skus,
    ARRAY_AGG(DISTINCT am.ArticleDescription ORDER BY am.ArticleDescription) AS descriptions
    --search_term2,
    --search_term3,
FROM step_one,
UNNEST(SPLIT(quoteline_sku, ",")) AS sku
LEFT JOIN `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_masterdata_view.dim_article_master_v` am 
    ON am.Article = sku 
WHERE booking_id = "WOW20003368"
AND ci_id = "CI-10120857"
AND search_term3 = "Self-Typed" 
AND am.SalesOrg = 1005
--WHERE LOWER(media_type) LIKE "%search%"
GROUP BY 1,2,3,4,5,6,7,8
ORDER BY media_start_date, ci_id
LIMIT 500
"""
).result()
check_df = check.to_dataframe()
check_df 

,booking_id,campaign_start_date,campaign_end_date,media_start_date,media_end_date,media_type,ci_id,search_term3,n_search_terms,search_terms,skus,descriptions
0,WOW20003368,2024-04-03,2024-04-30,2024-04-03,2024-04-09,Results Page Display,CI-10120857,Self-Typed,1104,"[2 litre full cream milk uht, 2 litre full cream milk | shop_all, 2 litre long life milk | shop_all, 30l water | shop_all, A4 paper, A4 paper | shop_all, APPLE DRINK | shop_all, Aero chocolate | shop_all, Aero | shop_all, Air | shop_all, Almonds | shop_all, Anguria, Antibacterial kids wipes | shop_all, Anusol | shop_all, Apple pie | shop_all, Apples | shop_all, Arnott’s gluten free | shop_all, Arrnots, Ashgrove | shop_all, Aspro | shop_all, Avocado | shop_all, BBQ shapes | shop_all, BROOM | shop_all, Baby Snacks | shop_all, Baby bottle | shop_all, Baby snacks | shop_all, Baby wipes | shop_all, Baking Soda | shop_all, Baking paper | shop_all, Balery Bread | shop_all, Bananas | shop_all, Bandaid | shop_all, Beans | shop_all, Beef mince | shop_all, Blue loo | shop_all, Blueberries | shop_all, Body wash | shop_all, Bodywash | shop_all, Bolognese | shop_all, Bottle cleaner | shop_all, Brauer | shop_all, Bread | shop_all, Bread loaf | shop_all, Bread thins | shop_all, Bread | shop_all, Broom | shop_all, Brooms | shop_all, Brush | shop_all, Brushes | shop_all, Bucket | shop_all, Buckets | shop_all, Burrito | shop_all, Butter chicken | shop_all, Butter | shop_all, CHIPS SNACKS | shop_all, CLEAN WIPES | shop_all, CLOTH | shop_all, COLESLAW PACK | shop_all, COLESLAW | shop_all, Cadbury egg | shop_all, Canaloni, Cannaloni, Canned corn | shop_all, Carrot | shop_all, Caster oil | shop_all, Caster sugar | shop_all, Cereal | shop_all, Cheese Slices mainland | shop_all, Cheese | shop_all, Chicken | shop_all, Chicken breast | shop_all, Chicken ham | shop_all, Chicken kieves | shop_all, Chicken mince | shop_all, Chilli oil | shop_all, Chocolate milk | shop_all, ClOths | shop_all, Clean Wipes | shop_all, Clean | shop_all, Cleaner | shop_all, Cleaners | shop_all, Cleaning & Maintenance | shop_all, Cleaning Cloth | shop_all, Cleaning brush | shop_all, Cleaning cloth | shop_all, Cleaning cloths | shop_all, Cleaning gloves | shop_all, Cleaning products | shop_all, Cleaning sponge | shop_all, Cleaning sponges | shop_all, Clinical | shop_all, Cloth | shop_all, Cloths | shop_all, Coca - Cola | shop_all, Coco | shop_all, Coconut body wash | shop_all, Coconut milk | shop_all, Coconut yoghurt | shop_all, Coffee | shop_all, Cold sore | shop_all, ...]","[151099, 151702, 278262]","[Drano Bathroom Gel 770ml, Drano Bathroom Max Gel 1.2 L, Drano Kitchen Gel 770ml]"


In [44]:
check = client.query(
"""
SELECT  
    SPLIT(udo_acs_stitch_id, ":")[OFFSET(0)] AS booking_id,
    SPLIT(udo_acs_stitch_id, ":")[OFFSET(3)] AS ci_id,
    udo_acs_stitch_id,
    COUNT(DISTINCT LOWER(TRIM(REPLACE(REGEXP_REPLACE(udo_search_keyword_5172,"[^a-zA-Z0-9 &-'@]", ""),'\t','')))) n_search_terms,
    ARRAY_AGG(DISTINCT LOWER(TRIM(REPLACE(REGEXP_REPLACE(udo_search_keyword_5172,"[^a-zA-Z0-9 &-'@]", ""),'\t',''))) IGNORE NULLS) AS search_terms
FROM `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_events` all_events
WHERE udo_acs_stitch_id LIKE "%WOW%"
AND DATE(post_time) = CURRENT_DATE("Australia/Sydney") - 1
GROUP BY 1,2,3
LIMIT 500
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,booking_id,ci_id,udo_acs_stitch_id,n_search_terms,search_terms
0,WOW20012568,CI-10172968,WOW20012568:wow-reach-grocery---app-banners:banner-category:CI-10172968:0015028:app,0,[]
1,WOW20009742,CI-10112306,WOW20009742:search-app-banner-2---bbq-meat:banner-category:CI-10112306:0057146:app,1,[british sausage]
2,WOW10001234,,WOW10001234:additional-search-display-banner:search-display-banner::0004274:app,0,[]
3,WOW20005406,CI-10077059,WOW20005406:wow-reach-all---app-banners:banner-category:CI-10077059:0045010:app,1,[prime glow berry]
4,"\\\""WOW20011299",CI-10158296,"\\\""WOW20011299:search-web-content-card---canned-food:content-card-search:CI-10158296:0015527:web\\\""",0,[]
5,WOW10033847,QL-0224628,WOW10033847:overlay-card--part-of-branded-shop:overlay-card:QL-0224628:0026344:web,0,[]
6,WOW20012415,CI-10146636,WOW20012415:search-web-banner---baby-edn:search-display-banner:CI-10146636:0014728:web,0,[]
7,WOW100413881,QL-0247135,WOW100413881:overlay-card-package:overlay-card:QL-0247135:0026711:web,0,[]
8,"\\\""WOW20013242",CI-10158155,"\\\""WOW20013242:half-price-search-content-card-part-of-roadblock:card-half-price:CI-10158155:0000000:web\\\""",0,[]
9,"\\\""WOW20012386",CI-10152039,"\\\""WOW20012386:search-web-content-card---canned-food:content-card-search:CI-10152039:0014750:web\\\""",0,[]


In [5]:
check = client.query(
"""
SELECT * FROM  `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns`  LIMIT 10
"""
).result()
check_df = check.to_dataframe()
check_df 

,booking_number,campaign_start_date,campaign_end_date,line_name,media_type,individual_product_string,media_start_date,media_end_date,total_media_spend
0,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446453,2023-10-11,2023-10-17,0.0
1,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446476,2023-10-11,2023-10-17,0.0
2,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446479,2023-10-11,2023-10-17,0.0
3,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,579716,2023-10-11,2023-10-17,0.0
4,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446487,2023-10-11,2023-10-17,0.0
5,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,579736,2023-10-11,2023-10-17,0.0
6,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,257713,2023-10-11,2023-10-17,0.0
7,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,479087,2023-10-11,2023-10-17,0.0
8,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446483,2023-10-11,2023-10-17,0.0
9,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446478,2023-10-11,2023-10-17,0.0


In [12]:
check = client.query(
"""


CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns`  AS (
    with step_one AS (
        SELECT
            booking_number,
            campaign_start_date,
            campaign_end_date,
            line_name,
            CASE
                WHEN media_location LIKE ('%TV%') THEN 'Mass_Social_Media'
                WHEN media_type IN (
                    'Counter Card',
                    'POC',
                    'Spotify',
                    'YouTube',
                    'Social',
                    'POC',
                    'Social - Wrapper',
                    'Build Capacity'
                ) THEN 'Mass_Social_Media'
                WHEN media_location IN ('Category', 'Aisle', 'Store', 'Screens', 'Emma Reach') THEN 'in-store'
                WHEN media LIKE ('%FB Post%') THEN 'Mass_Social_Media'
                WHEN media_type = 'Always On eDMs' THEN 'eDM'
                WHEN media_type = 'Single Send eDM' THEN 'eDM'
                WHEN media_type = 'BRANDED SHOP' THEN 'Branded Shop'
                WHEN media_type = 'CONTENT CARD' THEN 'Content Card'
                WHEN media_type = 'Digital Standalone Catalogue' THEN 'Catalogue'
                WHEN media_type = 'Display Recipes - Wrapper' THEN 'Display Recipes'
                WHEN media_type = 'RECIPE PACKAGE' THEN 'Display Recipes'
                WHEN media_type = 'Recipe Package' THEN 'Display Recipes'
                WHEN media_type = 'Standalone Catalogue - Double Spot' THEN 'Catalogue'
                WHEN media_type = 'Standalone Catalogue - Single Spot' THEN 'Catalogue'
                WHEN media_type = 'Standalone Catalogue- Third Page Vertical' THEN 'Catalogue'
                WHEN media_type = 'Catalogue Card' THEN 'Catalogue'
                WHEN media_type = 'BF Meal' THEN 'Others'
                WHEN media_type = 'Sampling' THEN 'Others'
                WHEN media_type = 'Bus Stop' THEN 'in-store'
                WHEN media_type = 'Display Promo Tile' THEN 'in-store'
                WHEN media_type = 'Aisle Fin' THEN 'in-store'
                WHEN media_type = 'Additional Production Fees' THEN 'Others'
                WHEN media_type = 'Event' THEN 'Others'
                WHEN media_type = 'National Radio' THEN 'Mass_Social_Media'
                WHEN media_type = 'Chilled Bus Stop & Decal Package' THEN 'in-store'
                WHEN media_type = 'National TV' THEN 'Mass_Social_Media'
                WHEN media_type = 'Press' THEN 'Mass_Social_Media'
                WHEN media_type = 'Content Card' THEN 'Others'
                WHEN media_type = 'Competition Management Fee' THEN 'Others'
                WHEN media_type = 'Prize pool' THEN 'Others'
                WHEN media_type = 'Chilled Bus Stop & Decal Package' THEN 'in-store'
                WHEN media_type = 'Chilled Fin Package' THEN 'in-store'
                WHEN media_type = 'Editorial' THEN 'Mass_Social_Media'
                WHEN media_type = 'BUS STOP' THEN 'in-store'
                WHEN media_type = 'Door Take' THEN 'in-store'
                WHEN media_type = 'Freezer Package' THEN 'in-store'
                WHEN media_type = 'PL - Mag' THEN 'Others'
                WHEN media_type = 'PL - Fresh Mag' THEN 'Others'
                WHEN media_type = 'PL - Demos' THEN 'Others'
                WHEN media_type = 'Pelmet' THEN 'in-store'
                WHEN media_type = 'Overlay Card' THEN 'in-store'
                WHEN media_type = 'Mass/Social Media' THEN 'Others'
                WHEN lower(media_type) like "%catalogue%" THEN 'Catalogue'
                WHEN LOWER(media_type) LIKE "%homepage%" THEN "Home Page"
                ELSE
                media_type
            END
            AS media_type,
            individual_product_string,
            MIN(media_start_date) AS media_start_date,
            MAX(media_end_date) AS media_end_date,
            SUM(quantity) * SUM(media_spend) AS total_media_spend

        FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns`,
        UNNEST(SPLIT(quoteline_sku, ',')) AS individual_product_string

        WHERE
            
            business_unit = 'Supermarket'
            AND booking_number IS NOT NULL
            AND quote_status NOT IN ('Not Approved','Draft','For Review','Sent')
            AND media_location NOT IN ('Category','Aisle','Store','Screens','Emma Reach')
            AND media_location NOT LIKE ('%TV%')
            AND media_type NOT IN ('Counter Card','POC','Spotify','YouTube','Social','POC','Social - Wrapper')
            AND (media_type <> 'Event')
            AND (media_location <> 'None' OR media LIKE ('%FB Post%') OR media LIKE ('%Branded%') OR media LIKE ('%Segments%'))
            AND media NOT IN ('Competition Management Fee','Additional Production Fees')
        AND media_start_date >= DATE("2023-10-01")
        AND media_end_date < CURRENT_DATE("Australia/Sydney")
        GROUP BY
            1,2,3,4,5,6
    ) 

    SELECT * 
    FROM step_one
    WHERE
        media_type NOT IN ('in-store','Others','eDM')
        AND media_type NOT LIKE ('%Woolworths Rewards%')
        AND media_type NOT LIKE ('%SAMPLING%') 
        AND media_type NOT LIKE "%Road Block%"
        AND media_type <> 'Radio'
        AND media_type <> 'Mass_Social_Media'
        AND LOWER(media_type) NOT LIKE "%branded shop%"

);
"""
).result()
check_df = check.to_dataframe()
check_df 

""


In [16]:
check = client.query(
"""
SELECT * FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` LIMIT 4
"""
).result()
check_df = check.to_dataframe()
check_df 

,quote_name,booking_number,business_unit,quote_status,quote_sku,net_amount,opportunity_owner,account,line_name,media,media_location,media_type,media_start_date,media_end_date,campaign_start_date,campaign_end_date,original_campaign_start_date,original_campaign_end_date,campaign_duration_weeks,store_list,quoteline_sku,revenue_sku,quantity,media_spend,duration_weeks,opportunity_name,campaign_objective,event_name,placement,online_department,brand,sku_item_name,quoteline_category,commercial_category,cnz_category_patch,line_name_all,original_booking_number,load_dttm,compliance_media_type
0,WOW20016829,WOW20016829,Supermarket,Stage 100%,None,22500.0,Hamid Alfughi,PZ Cussons Family Care (HHP) Deodorant (HBB),CI-10209521,Online Sampling,Online,Sampling,2024-11-06,2024-11-12,2024-11-06,2024-11-12,2024-11-06,2024-11-12,1,None,905268,905268,1.0,22500.0,1,Radiant -Online Sampling - $22.5k,Acquire New Customers,None,None,None,Radiant,None,None,None,None,"CI-10209521,",None,2024-08-12 10:20:57.219148,None
1,Q-00495,WOW10000496,Supermarket,Not Approved,None,40000.0,Hayley Clarke,Beam Suntory (DB),QL-0001009,eDM Themed,Orange Reports,Woolworths Rewards,NaT,NaT,NaT,NaT,NaT,NaT,<NA>,None,None,None,200000.0,0.0,<NA>,Nutriboost w2,None,None,None,None,None,None,None,None,None,QL-0001009,None,2024-08-12 10:20:57.219148,None
2,Q-00495,WOW10000496,Supermarket,Not Approved,None,40000.0,Hayley Clarke,Beam Suntory (DB),QL-0001011,eDM Band2 - Themed,Orange Reports,Woolworths Rewards,NaT,NaT,NaT,NaT,NaT,NaT,<NA>,None,None,None,1.0,40000.0,<NA>,Nutriboost w2,None,None,None,None,None,None,None,None,None,QL-0001011,None,2024-08-12 10:20:57.219148,None
3,Q-01160,WOW10001161,Supermarket,Not Approved,None,0.2,Hayley Clarke,Haleon Australia Pty Ltd,QL-0003369,eDM Always On,Orange Reports,Woolworths Rewards,NaT,NaT,NaT,NaT,NaT,NaT,<NA>,None,None,None,1.0,0.2,<NA>,GSK annual plan,Grow Existing Customers,None,None,None,None,None,None,None,None,QL-0003369,None,2024-08-12 10:20:57.219148,None


In [ ]:
check = client.query(
"""
"""
).result()
check_df = check.to_dataframe()
check_df 